#### Copy baseline recognition statistics from firestore to BigQuery

In [1]:
import datetime

In [6]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import db

cred = credentials.Certificate(r"C:\!Dev\projects\book_detection\keys\avea-biblosphere-210106-firebase-adminsdk-s3bjc-ff2c6ac275 (1).json")
firebase_admin.initialize_app(cred)

In [7]:
#firestore
db = firestore.client()

In [33]:
#bigquery
from google.oauth2 import service_account
from google.cloud import bigquery

project_id = 'biblosphere-210106'
credentials = service_account.Credentials.from_service_account_file('./keys/biblosphere-210106-46dbcd9058f7.json')

bq_client = bigquery.Client(credentials= credentials,project=project_id)

In [68]:
#create table
from google.cloud import bigquery

#client = bigquery.Client()

table_id = "biblosphere-210106.biblosphere.recognition_stats"

schema = [
    bigquery.SchemaField("timestamp", "NUMERIC", mode="REQUIRED"),
    bigquery.SchemaField("date", "DATETIME", mode="REQUIRED"),
    bigquery.SchemaField("photo_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("algorithm", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("known_books", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("duration_microsec", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("recognized_books", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("total_finded_books", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("detectron_finded_books", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("photo_url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("added", "DATETIME", mode="REQUIRED"),
]

table = bigquery.Table(table_id, schema=schema)
table = bq_client.create_table(table)  
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table biblosphere-210106.biblosphere.recognition_stats


In [70]:
def insert_photo_to_bigquery(doc, table_id):
    data = doc.to_dict()
    
    today_date = datetime.datetime(2021, 9, 10)
    today_timestamp = today_date.timestamp()
    algoritm = "Denis's baseline algorithm (not ml)"
    known_books = None
    
    record = [{u'timestamp': today_timestamp,
              u'date': today_date.isoformat(),
              u'photo_id': doc.id,
              u'algorithm': algoritm,
              u'known_books': known_books,
              u'duration_microsec': None,
              u'recognized_books': data['recognized'] if 'recognized' in data else 0,
              u'total_finded_books': data['total'] if 'total' in data else 0,
              u'detectron_finded_books': None,
              u'photo_url': data['thumbnail'] if 'thumbnail' in data else None,
              u'added': datetime.datetime.today().isoformat(),
    }]
    #print(record)
    #print(data['thumbnail'])
    #print()
    
    errors = bq_client.insert_rows_json(table_id, record)  # Make an API request.
    if errors != []:
        print("Encountered errors while inserting rows (photo_id: {}): {}".format(doc.id, errors))
    
    

In [71]:
#insert from firestore

table_id = "biblosphere-210106.biblosphere.recognition_stats"    
docs = db.collection('photos').stream()
examples = 9999

count = 0
for doc in docs:
    if count == examples:
        break
    insert_photo_to_bigquery(doc, table_id)
    count += 1
    #print('inserted:', count, doc.id)
    
print('inserted:', count)

inserted: 1275


In [65]:
def photo_in_bigquery(photo_id):
    query = f"SELECT photo_id FROM `biblosphere-210106.biblosphere.recognition_stats` where photo_id = '{photo_id}' "    
    query_job = bq_client.query(query)
    results = list(query_job.result())
    return len(results) != 0

In [ ]:
# inserted missed rows

table_id = "biblosphere-210106.biblosphere.recognition_stats"    
docs = db.collection('photos').stream()
examples = 9999

count = 0
for doc in docs:
    if count == examples:
        break
    if not photo_in_bigquery(doc.id):
        insert_photo_to_bigquery(doc, table_id)
    
    count += 1
    if count % 10 == 0:
        print(count)
    #print('inserted:', count, doc.id)

#### Check records in firestore, which not is inserted into BigQuery

In [22]:
collection = 'photos' 
res = db.collection(collection).where(u'recognition_stats.record_in_stats', u'==', False).stream()
count = 0
for doc in res:
    print('Doc:', doc.id)
    print(doc.to_dict())
    count += 1
print('\nfinded:', count)


finded: 0
